In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import networkx as nx
import pandas as pd

project = 'asymmetric-cove-187011' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

# Links Data


In [2]:
links = client.get_job('bquxjob_1ed98894_18f3aba0e77')
links_df = links.to_dataframe()

NotFound: 404 GET https://bigquery.googleapis.com/bigquery/v2/projects/asymmetric-cove-187011/queries/bquxjob_1ed98894_18f3aba0e77?maxResults=0&location=US&prettyPrint=false: Not found: Table asymmetric-cove-187011:_071415c1d8ba2a94b8bbe1fdc3579f265f9fcb3e.anon5b22dc85d34b9bd17fb370f2dc85fe469f197983832016150a646b665b89fb4b

Location: US
Job ID: bquxjob_1ed98894_18f3aba0e77


In [ ]:
print(links.query)

SELECT *
FROM
  `glossy-odyssey-366820.farcaster.links`;


In [ ]:
links_df.head()

,fid,timestamp,target_fid,type,deleted_at
0,495408,2024-04-21 10:01:42+00:00,4564,follow,NaT
1,315945,2024-02-07 11:03:04+00:00,1237,follow,NaT
2,354669,2024-03-29 14:39:48+00:00,416981,follow,NaT
3,404661,2024-03-30 23:40:37+00:00,320215,follow,NaT
4,9647,2023-04-13 12:16:16+00:00,11736,follow,NaT


In [ ]:
links_df.describe()

,fid,target_fid
count,73684211.0,73684211.0
mean,330747.487046,223662.478834
std,114356.803729,172206.456391
min,1.0,1.0
25%,268567.0,7143.0
50%,332036.0,268366.0
75%,418254.0,368835.0
max,512238.0,789012.0


# Profile Data

In [ ]:
profiles = client.get_job('bquxjob_59d5669f_18f3af35231')
profiles_df = profiles.to_dataframe()

In [ ]:
print(profiles.query)

SELECT *
FROM
  `glossy-odyssey-366820.farcaster.profiles`;


In [ ]:
profiles_df.head()

,fid,last_updated_at,data,custody_address
0,504692,2024-04-28 08:29:33+00:00,None,0x77b0352100ebdb86a45557296e05721e48cc6d01
1,511997,2024-05-02 16:45:35+00:00,None,0xed414b0b2d8ba9208c26fd264177f0afd7ab1c7f
2,506795,2024-04-30 04:30:49+00:00,None,0xf3cb1a6ccb27f19ed26a95e1e5cd3f6c0cf802d4
3,502808,2024-04-26 18:05:15+00:00,None,0x9f839c3f59d73582fa26197cf69ab7bc92a7120f
4,510407,2024-05-01 22:34:01+00:00,None,0xfabb0fa8d23643e7aa1e6fb0f13b0128e9c238d1


# Convert ~1/5 of the tabular data to graph data (Proof of concept!)

In [ ]:
G = nx.from_pandas_edgelist(
    links_df[0:40000000],
    source='fid',
    target='target_fid',
    create_using=nx.DiGraph()
)

# Run Pagerank on the fids




In [ ]:
pr = nx.pagerank(G)

# Process results to find top 10 fids by pagerank

In [ ]:
df_pr = pd.DataFrame.from_dict(pr,  orient='index', columns = ['pagerank']).reset_index(names=['fid'])

In [ ]:
df_pr

,fid,pagerank
0,495408,5.048934e-07
1,4564,4.046939e-04
2,315945,5.462764e-07
3,1237,1.926273e-03
4,354669,2.828531e-04
...,...,...
352517,477350,5.047068e-07
352518,174067,5.072136e-07
352519,371818,5.047068e-07
352520,244250,5.059765e-07


In [ ]:
top_10 = df_pr.sort_values(by='pagerank', ascending=False)[0:10]

In [ ]:
top10_profiles_df = pd.merge(profiles_df, top_10, on = 'fid').sort_values('pagerank', ascending=False)

In [ ]:
top10_profiles_df[['pagerank', 'fid', 'data']]

,pagerank,fid,data
8,0.006011,3,"{""bio"":""Working on Farcaster and Warpcast."",""d..."
9,0.005528,5650,"{""bio"":""hullo"",""display"":""Vitalik Buterin"",""pf..."
1,0.004993,99,"{""bio"":""@base contributor #001; onchain cities..."
0,0.004471,207,"{""bio"":""(summer of) protocol {support | guild ..."
7,0.004182,2,"{""bio"":""Technowatermelon. Elder Millenial. Bui..."
4,0.003750,12,"{""bio"":""Building @bountycaster (bountycaster.x..."
5,0.003712,347,"{""bio"":""I like building apps on web3 protocols..."
3,0.003654,129,"{""bio"":""Building /bright-moments - minting onc..."
2,0.003518,1325,"{""bio"":""BDFL of Quilibrium, Eng @ Farcaster, e..."
6,0.003439,368,"{""bio"":""I work for Ethereum at the Ethereum Fo..."
